In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

# Step 1: Load Data
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSwpoN3FMStDvEZmrG5-5goEPTTyEQrR5XGd3mcUou8lqKmLC44q2SMuuiL_1dGRcvuFVPSeB_gMvjs/pub?output=csv"
df = pd.read_csv(url)

# Step 2: Clean Data
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.iloc[:, :5]
df.columns = ['timestamp', 'location', 'temperature', 'humidity', 'carbon']
df[['lat', 'lng']] = df['location'].str.split(',', expand=True)
df['lat'] = df['lat'].astype(float)
df['lng'] = df['lng'].astype(float)

# Step 3: Create HeatMap Base
m = folium.Map(location=[df['lat'].mean(), df['lng'].mean()], zoom_start=12)

# Step 4: HeatMap Layers
for feature in ['temperature', 'humidity', 'carbon']:
    heat_data = [[row['lat'], row['lng'], row[feature]] for _, row in df.iterrows()]
    HeatMap(heat_data, radius=15, name=feature).add_to(m)

folium.LayerControl().add_to(m)

# Step 5: Predictions Using Linear Regression
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp')
df['time_ordinal'] = df['timestamp'].map(pd.Timestamp.toordinal)

predictions = {}
for feature in ['temperature', 'humidity', 'carbon']:
    X = df[['time_ordinal']]
    y = df[feature]
    model = LinearRegression()
    model.fit(X, y)

    future_dates = pd.date_range(df['timestamp'].max(), periods=5, freq='H')
    future_ordinals = future_dates.map(pd.Timestamp.toordinal).values.reshape(-1, 1)
    future_preds = model.predict(future_ordinals)

    predictions[feature] = list(zip(future_dates.strftime('%Y-%m-%d %H:%M'), future_preds))

# Step 6: Show Predictions
print("📈 Predictions for the next 5 hours:")
for feature in predictions:
    print(f"\n🔮 {feature.capitalize()} Prediction:")
    for timestamp, value in predictions[feature]:
        print(f"{timestamp} → {round(value, 2)}")

# Step 7: Save map
m.save("heatmap_with_predictions.html")
files.download("heatmap_with_predictions.html")


📈 Predictions for the next 5 hours:

🔮 Temperature Prediction:
2025-08-06 23:58 → 28.23
2025-08-07 00:58 → 27.16
2025-08-07 01:58 → 27.16
2025-08-07 02:58 → 27.16
2025-08-07 03:58 → 27.16

🔮 Humidity Prediction:
2025-08-06 23:58 → 53.3
2025-08-07 00:58 → 53.6
2025-08-07 01:58 → 53.6
2025-08-07 02:58 → 53.6
2025-08-07 03:58 → 53.6

🔮 Carbon Prediction:
2025-08-06 23:58 → 378.76
2025-08-07 00:58 → 382.2
2025-08-07 01:58 → 382.2
2025-08-07 02:58 → 382.2
2025-08-07 03:58 → 382.2


/tmp/ipython-input-1516480203.py:43: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_dates = pd.date_range(df['timestamp'].max(), periods=5, freq='H')
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/tmp/ipython-input-1516480203.py:43: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_dates = pd.date_range(df['timestamp'].max(), periods=5, freq='H')
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/tmp/ipython-input-1516480203.py:43: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_dates = pd.date_range(df['timestamp'].max(), periods

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>